In [ ]:
# Codigo para gerar os dados mocados

import json
import random
import datetime
import os
import math

output_path = "./data/turtle_sensors.jsonl"
os.makedirs("./data", exist_ok=True)

NUM_ROWS = 10_000

def generate_temperature(t):
    minute_of_day = t % 1440  
    daily_cycle = 0.5 * math.sin(2 * math.pi * (minute_of_day / 1440))

    long_term_rise = 4 * (t / NUM_ROWS)

    base_temp = random.uniform(17, 22)  # base for nights
    temp = base_temp + daily_cycle * 10 + long_term_rise

    return round(min(max(temp, 17), 34), 2)  # clamp between 17 and 34


def generate_movement(t):
    if t < NUM_ROWS * 0.75:
        movement_intensity = 0.02  # very calm
    elif t < NUM_ROWS * 0.9:
        movement_intensity = 0.08  # increasing
    else:
        movement_intensity = 0.4  # babies starting to break shells

    ax = random.uniform(-movement_intensity, movement_intensity)
    ay = random.uniform(-movement_intensity, movement_intensity)
    az = 1 + random.uniform(-movement_intensity, movement_intensity)  # gravity reference

    gx = random.uniform(-movement_intensity, movement_intensity)
    gy = random.uniform(-movement_intensity, movement_intensity)
    gz = random.uniform(-movement_intensity, movement_intensity)

    return (
        round(ax, 4), round(ay, 4), round(az, 4),
        round(gx, 4), round(gy, 4), round(gz, 4)
    )


with open(output_path, "w") as f:
    start_time = datetime.datetime.now()

    for t in range(NUM_ROWS):
        temp = generate_temperature(t)
        ax, ay, az, gx, gy, gz = generate_movement(t)

        reading = {
            "timestamp": (start_time + datetime.timedelta(seconds=t)).isoformat(),
            "temperature_c": temp,
            "accelerometer": {"ax": ax, "ay": ay, "az": az},
            "gyroscope": {"gx": gx, "gy": gy, "gz": gz}
        }

        f.write(json.dumps(reading) + "\n")

print(f"Generated {NUM_ROWS} sensor readings in:", output_path)

In [ ]:
%pip install -U dspy

In [ ]:
# Carregamento do corpus

import json
from sentence_transformers import SentenceTransformer

with open("./data/turtle_sensors.jsonl") as f:
    corpus = []
    for line in f:
        obj = json.loads(line)
        
        text = (
            f"timestamp: {obj['timestamp']}. "
            f"temperature: {obj['temperature_c']}C. "
            f"accelerometer ax={obj['accelerometer']['ax']}, "
            f"ay={obj['accelerometer']['ay']}, "
            f"az={obj['accelerometer']['az']}. "
            f"gyroscope gx={obj['gyroscope']['gx']}, "
            f"gy={obj['gyroscope']['gy']}, "
            f"gz={obj['gyroscope']['gz']}."
        )
        corpus.append(text)

print("Loaded", len(corpus), "sensor readings.")

In [ ]:

import dspy

# criação do embedder + retriever
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
embedder = dspy.Embedder(model.encode)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=10)


In [ ]:
# Configurando a LM

# %pip install python-dotenv
from dotenv import load_dotenv
import os

gemini_api_key = os.getenv("GEMINI_API_KEY")
anth_api_key = os.getenv("ANTHROPIC_API_KEY")

print("Loaded:", anth_api_key[:5] + "..." if anth_api_key else "Not found")

lm = dspy.LM("gemini/gemini-2.5-flash-lite", api_key=gemini_api_key)
dspy.configure(lm=lm)

In [ ]:
class SensorRAG(dspy.Module):
    def __init__(self):
        self.answer = dspy.ChainOfThought(
            "context, question -> response"
        )

    def forward(self, question):
        retrieved = search(question).passages
        return self.answer(context=retrieved, question=question)

sensor_rag = SensorRAG()

In [ ]:
# 1a Pergunta
question = "Qual foi a temperatura média?"
rag_output = sensor_rag(question=question)
llm_answer = rag_output.response

# recuperar documentos do RAG corretamente
retrieved_docs = search(question).passages   # <-- AGORA É LISTA DE STRINGS

print("\nDocumentos recuperados:", len(retrieved_docs))

In [ ]:
# 2a Pergunta
sensor_rag(question="Quando ocorreram picos de atividade do acelerômetro?")

In [ ]:
# 3a Pergunta
sensor_rag(question="Qual foi a temperatura máxima registrada?")

In [ ]:
#%pip install deepeval
%pip install anthropic

In [61]:
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric
)
from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from deepeval.models import GeminiModel, AnthropicModel, OllamaModel
import re

def rag_pipeline(question):
    rag_output = sensor_rag(question=question)
    llm_answer = rag_output.response
    retrieved_docs = search(question).passages
    print("\nRetrieved docs:", len(retrieved_docs))
    return llm_answer, retrieved_docs

# ---- PERGUNTA ----
question = "Qual foi a temperatura média?"

# ---- CALCULAR A RESPOSTA ESPERADA (GROUND TRUTH) ----
temps = []
for text in corpus:
    m = re.search(r"temperature: ([0-9.]+)C", text)
    if m:
        temps.append(float(m.group(1)))

expected_output = f"A temperatura média correta é {sum(temps)/len(temps):.3f}C"

# Criação dos casos de teste
saida_llm_1, contextos_saida = rag_pipeline(question)

test_case = LLMTestCase(
    input=question,
    actual_output=saida_llm_1,
    retrieval_context=contextos_saida,
    expected_output=expected_output
)

# ---- MODELO OPEN-SOURCE PARA AVALIAÇÃO ----
# eval_model = DeepEvalOllamaModel(model="llama3.1")
eval_model = OllamaModel("gpt-oss:120b-cloud")

# ---- MÉTRICAS ----
answer_relevancy = AnswerRelevancyMetric(model=eval_model)
contextual_precision = ContextualPrecisionMetric(model=eval_model)
contextual_recall = ContextualRecallMetric(model=eval_model)
contextual_relevancy = ContextualRelevancyMetric(model=eval_model)

# ---- EXECUTAR AVALIAÇÃO ----
results = evaluate(
    test_cases=[test_case],
    metrics=[answer_relevancy, contextual_precision, contextual_recall, contextual_relevancy]
)

# ---- RESULTADOS ----
print("\n--- RESULTADOS ---")
for r in results:
    print(r)



Retrieved docs: 10


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-oss:120b-cloud (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-oss:120b-cloud (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-oss:120b-cloud (Ollama), strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-oss:120b-cloud (Ollama), strict=False, 
async_mode=True)...



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-oss:120b-cloud (Ollama), reason: The score is 1.00 because the answer directly addressed the question with no irrelevant content., error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-oss:120b-cloud (Ollama), reason: The score is 1.00 because every node in the retrieval list is relevant—each node (ranks 1‑10) contains a temperature reading such as "temperature: 27.77C" and "temperature: 24.59C"—so there are no irrelevant nodes ranked above any relevant ones., error: None)
  - ❌ Contextual Recall (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-oss:120b-cloud (Ollama), reason: The score is 0.00 because sentence 1 ('A temperatura média correta é 21.721C') cannot be linked to any node in retrieval context; none of the retrieved nodes mention a temperature of 21.721C., error: None)
  - ✅ Contextual Relevancy (score

⚠ WARNING: No hyperparameters logged.
» ]8;id=567096;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 17.33s | token cost: 0.0 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.


--- RESULTADOS ---
('test_results', [TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the answer directly addressed the question with no irrelevant content.', strict_mode=False, evaluation_model='gpt-oss:120b-cloud (Ollama)', error=None, evaluation_cost=0.0, verbose_logs='Statements:\n[\n    "A temperatura média foi de 22.80°C."\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    }\n]'), MetricData(name='Contextual Precision', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because every node in the retrieval list is relevant—each node (ranks 1‑10) contains a temperature reading such as "temperature: 27.77C" and "temperature: 24.59C"—so there are no irrelevant nodes ranked above any relevant ones.', strict_mode=False, evaluation_model='gpt-oss:120b-cloud (Ollama)', error=None, evaluation_cost=0.0, verbose_logs='Verdi